## Environmental Setup

In [1]:
!pip install fugashi
!pip install ipadic
!pip install pretty_midi
!apt-get install fluidsynth
!wget "https://schristiancollins.com/generaluser.php?action=download&name=GeneralUserGSfMTV11.zip" -O soundfont.zip
!unzip soundfont.zip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=a16d62140470b8e4e15033678c9b4d136b65b80aff149e44acaa2452aa8f12e6
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.9 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=64d9ba8d2ca709165f6565b6228df9c92400be8acca4b381c7176dc4878889a4
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
Reading pa

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import time
import datetime
import random
import pretty_midi
import re
from music21 import converter, stream
import os
import shutil
from google.colab import files

## Transformer Models from Hugging Face

In [3]:
# Importing necessary modules from the transformers library
from transformers import AutoModelForSequenceClassification, BertJapaneseTokenizer, AutoTokenizer, pipeline

# Loading the pre-trained sentiment analysis model for Japanese with irony detection
model = AutoModelForSequenceClassification.from_pretrained("kit-nlp/bert-base-japanese-sentiment-irony")
# Loading the tokenizer associated with the selected model
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
# Creating a pipeline for sentiment analysis using the loaded model and tokenizer
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/110 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

## Creating folders for the output

In [4]:
# Define the song_name variable
# song_name = "YMCA"
# song_name = "when_will_my_life_began"
# CHANGE THE SONG NAME!
song_name = "can_you_feel_the_love_tonight"

genefolder_pr = f"A_chord/chord_pr/{song_name}/"
os.makedirs(genefolder_pr, exist_ok=True)
genefolder_sc = f"A_chord/score/{song_name}/"
os.makedirs(genefolder_sc, exist_ok=True)

midi_folder_p = f"B_chord_midi/piano/{song_name}/"
os.makedirs(midi_folder_p, exist_ok=True)
midi_folder_b = f"B_chord_midi/bass/{song_name}/"
os.makedirs(midi_folder_b, exist_ok=True)

output_dir = f"C_generated/{song_name}/"
os.makedirs(output_dir, exist_ok=True)

## User Input : Text (lylics) -> Chord

In [5]:
sentiment_data = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/NorikaNarimatsu/CC_final/main/chord_sentiment.csv'))
chord_info=[['C',{'Ⅰ':'C','Ⅱ':'D','Ⅲ':'E','Ⅳ':'F','Ⅴ':'G','Ⅵ':'A','Ⅶ':'B'}],['G',{'Ⅰ':'G','Ⅱ':'A','Ⅲ':'B','Ⅳ':'C','Ⅴ':'D','Ⅵ':'E','Ⅶ':'F#'}],
                  ['D',{'Ⅰ':'D','Ⅱ':'E','Ⅲ':'F#','Ⅳ':'G','Ⅴ':'A','Ⅵ':'B','Ⅶ':'C#'}],['A',{'Ⅰ':'A','Ⅱ':'B','Ⅲ':'C#','Ⅳ':'D','Ⅴ':'E','Ⅵ':'F#','Ⅶ':'G#'}],
                  ['E',{'Ⅰ':'E','Ⅱ':'F#','Ⅲ':'G#','Ⅳ':'A#','Ⅴ':'B','Ⅵ':'C#','Ⅶ':'D'}],['B',{'Ⅰ':'B','Ⅱ':'C#','Ⅲ':'D#','Ⅳ':'E','Ⅴ':'F#','Ⅵ':'G#','Ⅶ':'A#'}],
                  ['F#',{'Ⅰ':'F#','Ⅱ':'G#','Ⅲ':'A#','Ⅳ':'B','Ⅴ':'C#','Ⅵ':'D#','Ⅶ':'E#'}],['C#',{'Ⅰ':'C#','Ⅱ':'D#','Ⅲ':'E#','Ⅳ':'F#','Ⅴ':'G#','Ⅵ':'A#','Ⅶ':'B#'}],
                  ['F',{'Ⅰ':'F','Ⅱ':'G','Ⅲ':'A','Ⅳ':'Bb','Ⅴ':'C','Ⅵ':'D','Ⅶ':'E'}],['Bb',{'Ⅰ':'Bb','Ⅱ':'C','Ⅲ':'D','Ⅳ':'Eb','Ⅴ':'F','Ⅵ':'G','Ⅶ':'A'}],
                  ['Eb',{'Ⅰ':'Eb','Ⅱ':'F','Ⅲ':'G','Ⅳ':'Ab','Ⅴ':'Bb','Ⅵ':'C','Ⅶ':'D'}],['Ab',{'Ⅰ':'Ab','Ⅱ':'Bb','Ⅲ':'C','Ⅳ':'Db','Ⅴ':'Eb','Ⅵ':'F','Ⅶ':'G'}],
                  ['Db',{'Ⅰ':'Db','Ⅱ':'Eb','Ⅲ':'F','Ⅳ':'Gb','Ⅴ':'Ab','Ⅵ':'Bb','Ⅶ':'C'}],['Gb',{'Ⅰ':'Gb','Ⅱ':'Ab','Ⅲ':'Bb','Ⅳ':'Cb','Ⅴ':'Db','Ⅵ':'Eb','Ⅶ':'F'}],
                  ['Cb',{'Ⅰ':'Cb','Ⅱ':'Db','Ⅲ':'Eb','Ⅳ':'Fb','Ⅴ':'Gb','Ⅵ':'Ab','Ⅶ':'Bb'}]]

maj_data = pd.DataFrame(chord_info, columns=['key_table','chord_dic']).set_index('key_table')

In [6]:
def text2chord(key, Target_TXT):
    """
    Converts text input to chord progression based on sentiment analysis.

    Parameters:
    - key (str): The musical key for chord mapping.
    - Target_TXT (str): The input text for sentiment analysis and chord generation.

    Comments:
    - Utilizes a sentiment analysis model to determine the sentiment of the input text.
    - Maps sentiment scores to predefined chord progressions, considering the specified key.
    - Handles linguistic nuances for appropriate musical expression.
    - Generates a chord progression and saves it to a file along with the sentiment score.
    - Prints the original text and the resulting chord progression.

    """
    result = nlp(Target_TXT)

    # Adjust sentiment score based on label (negative or positive)
    num = result[0]['score'] * (-1 if result[0]['label'] == "ネガティブ" else 1)

    # Find the nearest chord based on sentiment score
    distance = np.abs(sentiment_data['score'] - num)
    indices = np.where(distance == np.min(distance))[0]
    text_list = sentiment_data['chord'][np.random.choice(indices)]

    # Map chords based on key and linguistic nuances
    no = maj_data.index.get_loc(key)
    dic = maj_data['chord_dic'][no]

    transed = []
    transed = [dic.get(moji, moji) for moji in text_list]
    chord_pr = ''.join(transed).lstrip(' ')
    chord_pr = chord_pr.replace("bC","B")
    chord_pr = chord_pr.replace("bD","Db")
    chord_pr = chord_pr.replace("bE","Eb")
    chord_pr = chord_pr.replace("bF","E")
    chord_pr = chord_pr.replace("bG","Gb")
    chord_pr = chord_pr.replace("bA","Ab")
    chord_pr = chord_pr.replace("bB","Bb")
    chord_pr = chord_pr.replace("E#","F")
    chord_pr = chord_pr.replace("B#","C")
    chord_pr = chord_pr.replace("C##","D")
    chord_pr = chord_pr.replace("D##","E")
    chord_pr = chord_pr.replace("F##","G")
    chord_pr = chord_pr.replace("G##","A")
    chord_pr = chord_pr.replace("A##","B")
    chord_pr = chord_pr.replace("Cbb","Bb")
    chord_pr = chord_pr.replace("Dbb","C")
    chord_pr = chord_pr.replace("Ebb","D")
    chord_pr = chord_pr.replace("Fbb","Eb")
    chord_pr = chord_pr.replace("Gbb","F")
    chord_pr = chord_pr.replace("Abb","G")
    chord_pr = chord_pr.replace("Bbb","A")
    chord_pr = chord_pr.replace("Cb#","C")
    chord_pr = chord_pr.replace("Db#","D")
    chord_pr = chord_pr.replace("Eb#","E")
    chord_pr = chord_pr.replace("Fb#","F")
    chord_pr = chord_pr.replace("Gb#","G")
    chord_pr = chord_pr.replace("Ab#","A")
    chord_pr = chord_pr.replace("Bb#","B")

    # Handle repetitions of chords
    if "la" not in Target_TXT and 0 <= chord_pr.count(" ") <= 2:
        chord_pr = chord_pr + " " + chord_pr

    # Save chord progression to file
    date_now = datetime.datetime.now()
    filename_ch = genefolder_pr + "chord_fw_" + date_now.strftime('%Y%m%d_%H%M%S%f')
    with open(filename_ch, "w", encoding="UTF-8") as txt:
        txt.write(chord_pr)

    # Save sentiment score to file
    filename_sc = genefolder_sc + "score_" + date_now.strftime('%Y%m%d_%H%M%S%f')
    with open(filename_sc, "w", encoding="UTF-8") as txt:
        txt.write(str(num))

    # Print original text and resulting chord progression and sentiment score
    print(Target_TXT, ":", chord_pr, "/", str(num))
    print("")


In [7]:
# Clean and create directories for chord progressions and sentiment scores
shutil.rmtree(genefolder_pr)
os.mkdir(genefolder_pr)

shutil.rmtree(genefolder_sc)
os.mkdir(genefolder_sc)

# Generate chord progressions based on user-specified keys and lyrics
# 'key': Choose from C, G, D, A, E, B, F#, C#, F, Bb, Eb, Ab, Db, Gb, Cb
# 'lyric': Enter "la+n" to display n random chord progressions or provide custom lyrics
# Note: Generating multiple chord progressions (la+n) may require multiple attempts

text2chord("D","愛を感じて")
text2chord("D","世界を包む　ハーモニー")
text2chord("D","命の歌よ")
text2chord("D","話したいけどどうすればいい")
text2chord("D","過去の真実ああだめだ")
text2chord("D","王はあなた")

愛を感じて : Cm7 Bm7 Bbm7 Eb / 0.8613585233688354

世界を包む　ハーモニー : G/B C D G D/F# / 0.7437107563018799

命の歌よ : G# G G# G / 0.744433581829071

話したいけどどうすればいい : G C#m7-5 F# G C#m7-5 F# / 0.686124861240387

過去の真実ああだめだ : Fm G Cm Bbm Eb / 0.5220361351966858

王はあなた : G A D G A D / 0.9682599306106567



## Machine input: Chord -> Midi

In [ ]:
def chord2midi_combined(lyric_chord, is_bass=False):
    # Chord and note mappings
    root = {'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 'E': 4, 'E#': 5, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7,
            'G#': 8, 'Ab': 8, 'A': -3, 'A#': -2, 'Bb': -2, 'B': -1}
    chord_type = {'': np.array([0, 4, 7]), 'm': np.array([0, 3, 7]), '7': np.array([0, 4, 12, 22]),
                  'm7': np.array([0, 3, 12, 22]), 'mM7': np.array([0, 3, 12, 23]), 'maj7': np.array([0, 4, 12, 23]),
                  'dim': np.array([0, 3, 6, 9, 12]), 'aug': np.array([0, 4, 8, 12]), 'add9': np.array([0, 4, 12, 26]),
                  'sus4': np.array([0, 5, 7, 12]), '7sus4': np.array([0, 5, 12, 22]), 'm6': np.array([0, 3, 12, 21]),
                  '6': np.array([0, 4, 12, 21]), 'm7-5': np.array([0, 3, 6, 12, 22]), '9': np.array([0, 4, 7, 22, 25])}
    on_chord = {'/Cb': 47, '/C': 48, '/C#': 49, '/Db': 49, '/D': 50, '/D#': 51, '/Eb': 51, '/E': 52, '/Fb': 52, '/E#': 53,
                '/F': 53, '/F#': 54, '/Gb': 54, '/G': 55, '/G#': 56, '/Ab': 56, '/A': 57, '/A#': 58, '/Bb': 58, '/B': 59}

    # Helper function to split chords
    def split_chord(chord):
        j = chord  # Original chord
        c = j      # Base note of the chord
        o = j      # Additional notes or modifiers
        if '/' in o:
            target = '/'
            idx = o.find(target)
            o = o[idx:]
        else:
            idx = ''
            o = ''
    # Process the base note and modifiers
        if len(c) > 1:
            if idx != '':
                c = c[0:2]
                # Handle sharp(#) or flat(b) modifiers
                if c[1] == '#' or c[1] == 'b':
                    c = c[0:2]
                    j = j[2:idx]
                else:
                    c = c[0:1]
                    j = j[1:idx]
            else:
                c = c[0:2]
                if c[1] == '#' or c[1] == 'b':
                    c = c[0:2]
                    j = j[2:]
                else:
                    c = c[0:1]
                    j = j[1:]
        else:
            j = '' # No base note, only modifiers
        return c, j, o

    # Initialize PrettyMIDI object and instrument based on is_bass parameter
    pm_combined = pretty_midi.PrettyMIDI(resolution=960, initial_tempo=120)
    if is_bass:
        instrument = pretty_midi.Instrument(48)
    else:
        instrument = pretty_midi.Instrument(0)
    # Read chords from file
    input_chords = open(os.path.expanduser(lyric_chord), 'r').read()
    chords = np.array(re.split(" +", input_chords.rstrip()))
    chords_len = len(chords)

    i = 0
    d_time_list = [0.125, 0.125, 0.125, 0.25, 0.25, 0.25, 0.5, 1.0, 1.0, 2.0]  # Chord intervals for melody
    d_time_b = 0.5  # Chord intervals for bass
    time = 0

    # Iterate through chords
    for chord in chords:
        i += 1
        croot, ctype, otype = split_chord(chord)

        d_time = float(random.choice(d_time_list))
        e_time = 2.0 - d_time
        # Set notes based on is_bass parameter
        if is_bass:
            notes = 36 + root[croot]
        else:
            notes = 60 + root[croot]
        # Adjust notes based on chord type and inversion
        if otype != "":
            if ctype in chord_type:
                notes += chord_type[ctype]
                notes = np.insert(notes, [0], on_chord[otype])
            else:
                notes += np.array([0, 12])
                notes = np.insert(notes, [0], on_chord[otype])
        else:
            if ctype in chord_type:
                notes += chord_type[ctype]
            else:
                notes += np.array([0, 12])
        # Add notes to the instrument
        if i == chords_len:
            d_time = 2.0
            for note_number in notes:
                note = pretty_midi.Note(velocity=100, pitch=note_number, start=time, end=time + d_time)
                instrument.notes.append(note)
        else:
            for note_number in notes:
                note = pretty_midi.Note(velocity=80, pitch=note_number, start=time, end=time + d_time)
                instrument.notes.append(note)
            for e_number in range(int(e_time / d_time + 1)):
                note = pretty_midi.Note(velocity=0, pitch=0, start=time, end=time + d_time)
                instrument.notes.append(note)
                time = time + d_time
    # Add the instrument to the PrettyMIDI object
    pm_combined.instruments.append(instrument)
    # Save the generated MIDI file based on is_bass parameter
    if is_bass:
        filepath = midi_folder_b + f"chords_{song_name}_{l}.mid"
    else:
        filepath = midi_folder_p + f"chords_{song_name}_{l}.mid"

    pm_combined.write(filepath)


In [ ]:
shutil.rmtree(midi_folder_p)
os.mkdir(midi_folder_p)
shutil.rmtree(midi_folder_b)
os.mkdir(midi_folder_b)

file_paths = [os.path.join(genefolder_pr, file) for file in os.listdir(genefolder_pr) if file != '.DS_Store']
file_paths.sort(key=os.path.getmtime)
l=0
for chord_pr in file_paths:
    l += 1
    chord2midi_combined(chord_pr, is_bass= True)
    chord2midi_combined(chord_pr, is_bass= False)

In [ ]:
# remove the contents if we have any
shutil.rmtree(output_dir)
os.mkdir(output_dir)

# get all files in the folders
file_paths = [os.path.join(genefolder_pr, file) for file in os.listdir(genefolder_pr) if file != '.DS_Store']
midi_paths = [os.path.join(midi_folder_p, file) for file in os.listdir(midi_folder_p) if file != '.DS_Store']
# sort them based on the time
file_paths.sort(key=os.path.getmtime)
midi_paths.sort(key=os.path.getmtime)

## Combining lines and Final Output

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Create a Stream to store the concatenated sounds
output_stream = stream.Score()

# Enumerate through the generated MIDI files
for i, chord_pr in enumerate(file_paths):
    # Load the MIDI file for piano and bass and Parse piano and bass MIDI files using music21 converter
    piano_midi_path = os.path.join(midi_folder_p, f'chords_{song_name}_{i + 1}.mid')
    bass_midi_path = os.path.join(midi_folder_b, f'chords_{song_name}_{i + 1}.mid')
    piano_midi_stream = converter.parse(piano_midi_path)
    bass_midi_stream = converter.parse(bass_midi_path)

    # Insert piano and bass MIDI streams into the output stream
    output_stream += piano_midi_stream
    output_stream += bass_midi_stream

# Store the concatenated sounds as a new MIDI file in the output_dir
combined_output_path = os.path.join(output_dir, f'combined_sounds_{song_name}.mid')
output_stream.write('midi', fp=combined_output_path)

# Show the combined output stream
output_stream.show('midi')

# Extract chords from the combined output stream
extracted_chords = []
for element in output_stream.flat:
    if 'Chord' in element.classes:
        chord_pitches = [str(p) for p in element.pitches]
        chord_name = f"{element.root()} {element.quality} {', '.join(chord_pitches)}"
        extracted_chords.append(chord_name)

# Print the extracted chords
print("Extracted Chords:")
for i, chord in enumerate(extracted_chords):
    print(f"Chord {i + 1}: {chord}")


Extracted Chords:
Chord 1: B3 minor B3, D4, F#4
Chord 2: A3 major A3, C#4, E4
Chord 3: A3 major A3, C#4, E4
Chord 4: E4 major E4, G#4, B4
Chord 5: F4 other C#4, F4, G#4
Chord 6: G4 minor G4, B-4, D5
Chord 7: B3 minor B3, D4, F#4
Chord 8: B-3 other B-3, C#4, F4
Chord 9: E-4 other B3, E-4, F#4
Chord 10: D4 major D4, F#4, D5, C#6
Chord 11: B-4 other F#4, B-4, C#5
Chord 12: E-4 major E-4, G4, B-4
Chord 13: E-4 major E-4, G4, B-4
Chord 14: B-3 other B-3, C#4, F4
Chord 15: A3 minor A3, C4, E4
Chord 16: F#4 diminished C4, E-4, F#4, A4, C5
Chord 17: E4 major E4, G#4, E5, D6
Chord 18: E4 major E4, G#4, B4
Chord 19: F4 other C#4, F4, G#4
Chord 20: G4 minor G4, B-4, D5
Chord 21: A3 minor A3, C4, E4
Chord 22: C#4 minor C#4, E4, G#4
Chord 23: C#4 minor C#4, E4, G#4
Chord 24: B-4 other F#4, B-4, C#5
Chord 25: E-4 major E-4, G4, B-4
Chord 26: E-4 major E-4, G4, B-4


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Convert the combined MIDI file to WAV using FluidSynth and pydub
soundfont_path = "GeneralUserGSfMTV11.sf2"
wav_output_path = os.path.join(output_dir, f'combined_sounds_{song_name}.wav')
midi_output_path = os.path.join(output_dir, f'combined_sounds_{song_name}.midi')

# Convert MIDI to WAV using FluidSynth
!fluidsynth -ni "{soundfont_path}" "{combined_output_path}" -F "{wav_output_path}"

# Download the WAV file from Colab
files.download(wav_output_path)


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter 'GeneralUserGSfMTV11.sf2' not a SoundFont or MIDI file or error occurred identifying it.
Rendering audio to file '03_generated/when_will_my_life_began/combined_sounds_when_will_my_life_began.wav'..


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### References:

* https://ichi.pro/furosutosongu-ai-o-shiyoshite-shi-kara-ongaku-o-seiseisuru-109320446959370
* transformers:https://github.com/huggingface/transformers
* https://qiita.com/hnishi/items/0d32a778e375a99aff13
* https://www.ufret.jp/
* https://qiita.com/kinopee0120/items/7bbcda4afa7e501d6272
* https://qiita.com/a2kiti/items/c0fea6e415bf7d743e16